# 重试

In [ ]:
# from rtsp.rtsp_server import setup_server
#
# setup_server()

from retrying import retry
import random
import time


def retry_if_error(exception):
    return exception.__class__ in {ValueError, TimeoutError}


def retry_if_not_result(result):
    if result in [5, 10]:
        print("result success {}".format(result))
        return False
    else:
        print('result retry {}'.format(result))
        return True


@retry(stop_max_attempt_number=50, wait_exponential_multiplier=1000, wait_exponential_max=60000,
       retry_on_exception=retry_if_error)
def foo():
    print(int(time.time()), end=' ')
    a = random.randint(1, 15)
    if a >= 2:
        print('Error {}'.format(a))
        raise ValueError
    print('success', a)


@retry(retry_on_result=retry_if_not_result)
def foo2():
    print(int(time.time()), end=' ')
    a = random.randint(1, 15)
    return a


# print(foo2())
foo()


# 超时

实际项目中会涉及到需要对有些函数的响应时间做一些限制，如果超时就退出函数的执行，停止等待。

可以利用python中的装饰器实现对函数执行时间的控制。

## Unix 信号机制

**限制：需要在linux系统上，并且必须在主线程中使用**

In [ ]:
# 有回调函数的超时报错装饰器
import signal
import time


def set_timeout(num, callback):
    def wrap(func):
        def handle(
                signum, frame
        ):  # 收到信号 SIGALRM 后的回调函数，第一个参数是信号的数字，第二个参数是the interrupted stack frame.
            raise RuntimeError

        def to_do(*args, **kwargs):
            try:
                signal.signal(signal.SIGALRM, handle)  # 设置信号和回调函数
                signal.alarm(num)  # 设置 num 秒的闹钟
                print('start alarm signal.')
                r = func(*args, **kwargs)
                print('close alarm signal.')
                signal.alarm(0)  # 关闭闹钟
                return r
            except RuntimeError as e:
                callback()

        return to_do

    return wrap


if __name__ == '__main__':

    def after_timeout():  # 超时后的处理函数
        print("函数超时")

    @set_timeout(2, after_timeout)  # 限时 2 秒
    def connect():  # 要执行的函数
        time.sleep(4)  # 函数执行时间，写大于2的值，可测试超时
        print('函数正常执行')

    connect()

## func_timeout

>pip install func_timeout

In [1]:
import time

from func_timeout import func_set_timeout, FunctionTimedOut


@func_set_timeout(5)
def f():
    while True:
        print("1")
        time.sleep(1)


if __name__ == '__main__':
    try:
        f()
    except FunctionTimedOut:
        print("timeout!")

1
1
1
1
1
timeout!


## eventlet

>pip install eventlet

In [7]:
import requests
import eventlet
import time

eventlet.monkey_patch()

time_limit = 3  #set timeout time 3s
print(int(time.time()))
with eventlet.Timeout(time_limit, False):
    time.sleep(5)
    r = requests.get("https://me.csdn.net/dcrmg", verify=False)
    print('ok')
print(int(time.time()))
print('over')

1588942588
1588942591
over


## Timer

- https://www.cnblogs.com/xueweihan/p/6653610.html

In [17]:
from threading import Timer
import time


def set_timeout(num):
    def wrap(func):
        def stop():
            print(int(time.time()))
            raise RuntimeError

        def to_do(*args, **kwargs):
            # n 秒后执行
            t = Timer(num, stop)
            t.start()
            r = func(*args, **kwargs)
            t.cancel()
            return r

        return to_do

    return wrap


print(int(time.time()))
try:

    @set_timeout(4)
    def foo():
        time.sleep(6)
except RuntimeError as e:
    print('e', e)
foo()

1588943662
1588943666


Exception in thread Thread-5:
Traceback (most recent call last):
  File "D:\SoftInstall\Anacoda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "D:\SoftInstall\Anacoda3\lib\threading.py", line 1166, in run
    self.function(*self.args, **self.kwargs)
  File "<ipython-input-17-eecf61e36e1b>", line 9, in stop
    raise RuntimeError
RuntimeError

